### Упражнение 1

* Скачайте с портала Открытых Данных производственный календарь России: https://data.gov.ru/opendata/7708660670-proizvcalendar
* На его основе определите соответствующий календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`
* С помощью полученного в пункте выше календаря и модуля `pd.offsets` создайте DataFrame one-hot календарных признаков:
    * День недели
    * Месяц
    * Выходной день
    * Праздничный день
    * Предпраздничный рабочий день
    * Последний день месяца
    * Последний рабочий день месяца
    * Предновогодний выходной день

In [ ]:
import csv

with open('/content/data-20191112T1252-structure-20191112T1247.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [ ]:
import pandas as pd
import numpy as np

calendar = pd.read_csv('/content/data-20191112T1252-structure-20191112T1247.csv')
calendar

,Год/Месяц,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Всего рабочих дней,Всего праздничных и выходных дней,Количество рабочих часов при 40-часовой рабочей неделе,Количество рабочих часов при 36-часовой рабочей неделе,Количество рабочих часов при 24-часовой рабочей неделе
0,1999,"1,2,3,4,6*,7,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,24,25,30*","1,2,3,4,8,9,10,15,16,22,23,29,30","5,6,11*,12,13,14,19,20,26,27","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31","6,7,8,13,14,20,21,27,28","4,5,11,12,13,18,19,25,26,31*",251,114,2004,1807.2,1204.8
1,2000,"1,2,3,4,6*,7,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","4,5,7*,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","1,2,6,7,8*,9,13,14,20,21,27,28","3,4,10,11,12,17,18,24,25","1,2,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30","1,7,8,14,15,21,22,28,29","4,5,7,11,12,18,19,25,26","2,3,9,10,11*,12,16,17,23,24,30,31",250,116,1995,1800.0,1200.0
2,2001,"1,2,6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,24,25","3,4,7*,8,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29,30*","1,2,5,6,8*,9,12,13,19,20,26,27","2,3,9,10,11*,12,16,17,23,24,30","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","6,7,13,14,20,21,27,28","3,4,6*,7,10,11,17,18,24,25","1,2,8,9,12,15,16,22,23,29,30",251,114,2001,1807.2,1204.8
3,2002,"1,2,5,6,7,12,13,19,20,26,27","2,3,9,10,16,17,22*,23,24,25","2,3,7*,8,9,10,16,17,23,24,30,31","6,7,13,14,20,21,28,30*","1,2,3,4,5,8*,9,10,11,12,19,25,26","1,2,8,9,11*,12,15,16,22,23,29,30","6,7,13,14,20,21,27,28","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","5,6,12,13,19,20,26,27","2,3,6*,7,8,9,16,17,23,24,30","1,7,8,11*,12,13,14,21,22,28,29,31*",250,115,1992,1792.0,1192.0
4,2003,"1,2,3,5*,6,7,11,12,18,19,25,26","1,2,8,9,15,16,22,23,24","1,2,7*,8,9,10,15,16,22,23,29,30","5,6,12,13,19,20,26,27,30*","1,2,3,4,8*,9,10,11,17,18,24,25,31","1,7,8,11*,12,13,14,15,22,28,29","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","4,5,11,12,18,19,25,26","1,2,6*,7,8,9,15,16,22,23,29,30","6,7,11*,12,13,14,20,21,27,28,31*",250,115,1992,1792.0,1192.0
5,2004,"1,2,3,4,6*,7,10,11,17,18,24,25,31","1,7,8,14,15,21,22,23,28,29","6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,24,25,30*","1,2,3,4,8,9,10,15,16,22,23,29,30","5,6,11*,12,13,14,19,20,26,27","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31","6,7,8,13,14,20,21,27,28","4,5,11,12,13,18,19,25,26,31*",251,115,2004,1803.2,1200.8
6,2005,"1,2,3,4,5,6,7,8,9,10,15,16,22,23,29,30","5,6,12,13,19,20,22*,23,26,27","5*,6,7,8,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30","1,2,7,8,9,14,15,21,22,28,29","4,5,11,12,13,18,19,25,26","2,3,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","3,4,10,11,17,18,24,25","1,2,8,9,15,16,22,23,29,30","3*,4,5,6,12,13,19,20,26,27","3,4,10,11,17,18,24,25,31",248,117,1981,1782.6,1187.4
7,2006,"1,2,3,4,5,6,7,8,9,14,15,21,22,28,29","4,5,11,12,18,19,22*,23,24,25","4,5,7*,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","1,6*,7,8,9,13,14,20,21,27,28","3,4,10,11,12,17,18,24,25","1,2,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30","1,7,8,14,15,21,22,28,29","3*,4,5,6,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31",248,117,1981,1782.6,1187.4
8,2007,"1,2,3,4,5,6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,22*,23,24,25","3,4,7*,8,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28*,29,30","1,5,6,8*,9,12,13,19,20,26,27","2,3,9*,10,11,12,16,17,23,24,30","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","6,7,13,14,20,21,27,28","3,4,5,10,11,17,18,24,25","1,2,8,9,15,16,22,23,29*,30,31",249,116,1986,1786.8,1189.2
9,2008,"1,2,3,4,5,6,7,8,12,13,19,20,26,27","2,3,9,10,16,17,22*,23,24,25","1,2,7*,8,9,10,15,16,22,23,29,30","5,6,12,13,19,20,26,27,30*","1,2,3,8*,9,10,11,17,18,24,25,31","1,8,11*,12,13,14,15,21,22,28,29","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23

In [ ]:
free_days = [[['{}-{}-{}'.format(v, j, q) for q in calendar.iloc[i,j+1].split(',') if q[-1]!='*'] \
               for j in range(12)] \
               for i, v in enumerate(calendar.iloc[15:,0])]

In [ ]:
free_days = np.array(free_days).flatten()
list_free_days = []
for i in range(27):
  list_free_days.extend(free_days[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [ ]:
less_work_days = np.array([[['{}-{}-{}'.format(v, j, q) for q in calendar.iloc[i,j+1].split(',') if q[-1]=='*'] \
                    for j in range(12)] \
                    for i, v in enumerate(calendar.iloc[15:,0])], dtype=object)

less_work_days = less_work_days.flatten()

less_work_days = [less_work_days[i] \
                  for i in range(len(less_work_days)) \
                  if (less_work_days[i]!=list())]

less_work_days = [less_work_days[i][0][:-1] \
                  for i in range(len(less_work_days))]

In [ ]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, EasterMonday
from pandas.tseries.offsets import DateOffset, Day
import datetime

# Он НА ОСНОВЕ календаря, но не копирует 1 в 1
class RussianBusinessCalendar(AbstractHolidayCalendar):
    start_date = datetime.datetime(2014, 1, 1)
    end_date = datetime.datetime(2025, 12, 31)
    rules = [Holiday(name = 'New Year', day = 1, month = 1),
             Holiday(name = 'New Year1', day = 2, month = 1),
             Holiday(name = 'New Year2', day = 3, month = 1),
             Holiday(name = 'New Year3', day = 4, month = 1),
             Holiday(name = 'New Year4', day = 5, month = 1),
             Holiday(name = 'Christmas', day = 7, month = 1),
             Holiday(name = '23 Febr', day = 23, month = 2),
             Holiday(name = '8 March', day = 8, month = 3),
             Holiday(name = '1 May', day = 1, month = 5),
             Holiday(name = '9 May', day = 1, month = 9),
             Holiday(name = '12 June', day = 12, month = 6),
             Holiday(name = '4 November', day = 1, month = 1)]

russian_busday = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri', 
                                              calendar=RussianBusinessCalendar())

In [ ]:
holidays_list = [1 if i in RussianBusinessCalendar().holidays(start = datetime.datetime(2014, 1, 1), end = datetime.datetime(2025, 12, 31)) else 0 \
                 for i in pd.date_range('2014-01-01', '2025-12-31', freq='D')]

In [ ]:
before_holiday = [1 if i in (RussianBusinessCalendar().holidays(start = datetime.datetime(2014, 1, 1), end = datetime.datetime(2025, 12, 31)) - russian_busday) else 0 \
                 for i in pd.date_range('2014-01-01', '2025-12-31', freq='D')]

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimes.py:694: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


In [ ]:
month_end = [1 if i in (pd.date_range('2014-01-01', '2025-12-31', freq='D') + pd.tseries.offsets.MonthEnd()).drop_duplicates() else 0 \
             for i in pd.date_range('2014-01-01', '2025-12-31', freq='D')]

In [ ]:
BM = [(pd.date_range('2014-01-01', '2025-12-31', freq='D') + 
      pd.tseries.offsets.CustomBusinessMonthEnd(weekmask='Mon Tue Wed Thu Fri', calendar=RussianBusinessCalendar())).drop_duplicates()]

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimes.py:694: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


In [ ]:
bus_month_end = [1 if (i in BM[0]) else 0 for i in pd.date_range('2014-01-01', '2025-12-31', freq='D')]

In [ ]:
Index = pd.date_range('2014-01-01', '2025-12-31', freq='D')

In [ ]:
new_years = pd.date_range('2013-01-01', periods = 12, freq='Y') + pd.tseries.offsets.Day()

In [ ]:
def closest_one(i, c):
  i-=1
  while True:
    if c[i] == 1:
      return i
    else:
      i-=1

In [ ]:
prev_day_off=[]
for k in range(len(new_years)):
  for e, i in enumerate(Index):
    if new_years[k] == i:
      prev_day_off.append(Index[closest_one(e, day_off)])

In [ ]:
before_new_year = [1 if (i in prev_day_off) else 0 for i in pd.date_range('2014-01-01', '2025-12-31', freq='D')]

In [ ]:
day_off = [int(i.dayofweek == 5 or i.dayofweek == 6 or holidays_list[e]) for e, i in enumerate(Index)]

In [ ]:
events = pd.DataFrame(data = {'выходной': day_off,
                              'праздник': holidays_list,
                              'предпраздничный рабочий день': before_holiday,
                              'последний день месяца': month_end,
                              'последний рабочий день месяца': bus_month_end,
                              'пн': [int(i.dayofweek == 0) for i in Index],
                              'вт': [int(i.dayofweek == 1) for i in Index],
                              'ср': [int(i.dayofweek == 2) for i in Index],
                              'чт': [int(i.dayofweek == 3) for i in Index],
                              'пт': [int(i.dayofweek == 4) for i in Index],
                              'сб': [int(i.dayofweek == 5) for i in Index],
                              'вс': [int(i.dayofweek == 6) for i in Index],
                              'январь': [int(i.month == 1) for i in Index],
                              'февраль': [int(i.month == 2) for i in Index],
                              'март': [int(i.month == 3) for i in Index],
                              'апрель': [int(i.month == 4) for i in Index],
                              'май': [int(i.month == 5) for i in Index],
                              'июнь': [int(i.month == 6) for i in Index],
                              'июль': [int(i.month == 7) for i in Index],
                              'август': [int(i.month == 8) for i in Index],
                              'сентябрь': [int(i.month == 9) for i in Index],
                              'октябрь': [int(i.month == 10) for i in Index],
                              'ноябрь': [int(i.month == 11) for i in Index],
                              'декабрь': [int(i.month == 12) for i in Index],
                              'Предновогодний выходной день': before_new_year}, 
                      index = Index)

In [ ]:
events

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2014-01-01,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-02,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-03,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-04,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-05,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-27,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2025-12-28,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1
2025-12-29,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2025-12-30,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
# пример требуемого DataFrame 
events

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2014-01-01,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-02,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-03,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-04,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-05,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


### Упражнение 2

* Для временного ряда курса акций Сбербанка на основе производственного календаря России сформируйте DataFrame признаков:
    * Лаговые переменные рабочих дней с глубинами: 1, 3, 5
    * Скользящие статистики `['mean', 'median', 'max', 'min', 'std'] ` с окнами 1, 3, 5 сгрупированные по дням недели
    * Статистики `['mean', 'median', 'max', 'min', 'std'] ` в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням

In [ ]:
stocks = pd.read_csv('/content/sber_stocks.csv', usecols=['CLOSE', 'TRADEDATE'], 
                 index_col='TRADEDATE', parse_dates=True)
stocks

,CLOSE
TRADEDATE,
2013-03-25,98.79
2013-03-26,97.20
2013-03-27,96.75
2013-03-28,98.59
2013-03-29,98.76
...,...
2020-01-03,255.00
2020-01-06,253.90
2020-01-08,259.15


In [ ]:
# Делаю в предположении, что расширяющееся - значит 1,3,5, а группировать по дням недели надо по всем вместе и сразу
stocks_shift_1 = stocks.shift(1, freq = russian_busday)
stocks_shift_3 = stocks.shift(3, freq = russian_busday)
stocks_shift_5 = stocks.shift(5, freq = russian_busday)

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimes.py:694: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  PerformanceWarning,


In [ ]:
s_and_e = stocks.join(events).reindex(stocks.index[197:])

In [ ]:
s_and_e_shift_1 = s_and_e.groupby(by = ['пн', 'вт', 'ср', 'чт', 'пт']).rolling(window=1).agg(['mean', 'median', 'max', 'min', 'std'])
s_and_e_shift_3 = s_and_e.groupby(by = ['пн', 'вт', 'ср', 'чт', 'пт']).rolling(window=1).agg(['mean', 'median', 'max', 'min', 'std'])
s_and_e_shift_5 = s_and_e.groupby(by = ['пн', 'вт', 'ср', 'чт', 'пт']).rolling(window=1).agg(['mean', 'median', 'max', 'min', 'std'])

In [ ]:
s_and_e_shift_last_1 = s_and_e.groupby(by = ['пн', 'вт', 'ср', 'чт', 'пт', 'праздник', 'предпраздничный рабочий день']).rolling(window=1).agg(['mean', 'median', 'max', 'min', 'std'])
s_and_e_shift_last_3 = s_and_e.groupby(by = ['пн', 'вт', 'ср', 'чт', 'пт', 'праздник', 'предпраздничный рабочий день']).rolling(window=3).agg(['mean', 'median', 'max', 'min', 'std'])
s_and_e_shift_last_5 = s_and_e.groupby(by = ['пн', 'вт', 'ср', 'чт', 'пт', 'праздник', 'предпраздничный рабочий день']).rolling(window=5).agg(['mean', 'median', 'max', 'min', 'std'])